<a href="https://colab.research.google.com/github/BerkBelhan/Large-Language-Models/blob/main/Company_Database_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Homework:** Developing a Chatbot using VS Code / Colab, SQLite, Gemini API, and Gradio

# Install Dependencies

In [ ]:
!pip install google-generativeai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00


# Initial Configuration

In [ ]:
import os
import json
import sqlite3
import google.generativeai as genai
from google.colab import userdata
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Configure the API key
key = userdata.get('GOOGLE_API_KEY')

# Configure the API key (Replace with your actual key)
genai.configure(api_key = key)

# Configure the model
generation_config = {
    "temperature": 0.1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "application/json",
}

# System Prompt with Structured Output

In [ ]:


# SystemPrompt v1.3
system_prompt = """
You are a helpful assistant for a company database. Your goal is to understand the manager's intent and convert it into a valid SQL query to retrieve information
from the Northwind database and then using database output provide user friendly output about the data. Therefore, there will be 2 different output type:
SQL Query and User Friendly response. You have to provide the best information about the context. The user may ask questions in Turkish or English, and your responses
 should be in the appropriate language.

**Northwind Database Schema:**
Table: Categories
Columns: CategoryID (INTEGER), CategoryName (TEXT), Description (TEXT)
Table: Customers
Columns: CustomerID (INTEGER), CustomerName (TEXT), ContactName (TEXT), Address (TEXT), City (TEXT), PostalCode (TEXT), Country (TEXT)
Table: Employees
Columns: EmployeeID (INTEGER), LastName (TEXT), FirstName (TEXT), BirthDate (DATE), Photo (TEXT), Notes (TEXT)
Table: Shippers
Columns: ShipperID (INTEGER), ShipperName (TEXT), Phone (TEXT)
Table: Suppliers
Columns: SupplierID (INTEGER), SupplierName (TEXT), ContactName (TEXT), Address (TEXT), City (TEXT), PostalCode (TEXT), Country (TEXT), Phone (TEXT)
Table: Products
Columns: ProductID (INTEGER), ProductName (TEXT), SupplierID (INTEGER), CategoryID (INTEGER), Unit (TEXT), Price (NUMERIC)
Table: Orders
Columns: OrderID (INTEGER), CustomerID (INTEGER), EmployeeID (INTEGER), OrderDate (DATETIME), ShipperID (INTEGER)
Table: OrderDetails
Columns: OrderDetailID (INTEGER), OrderID (INTEGER), ProductID (INTEGER), Quantity (INTEGER)


**Example:**
**Input:** "How many customers are there ?"
**Output:** "SELECT COUNT(*) AS total_customers FROM customers;"
**Input:** "Current overall result is '159'"
**Output:** "In our firm there are 159 customers."

**Example2:**
**Input:** "Provide the details about product has id 12"
**Output:** "SELECT * FROM Products;"
**Input:** "Current overall result is 'etc etc...'"
**Output:** "SELECT * FROM ProductDetails;"
**Input:** "Current overall result is "{composite_version_of_two_request}"
**Output:** "Here is the all the informations about product 12\n Name: etc\nLocation: etc..."

**Example3:**
**Input:** "How many customers are there ?"
**Output:** "SELECT COUNT(*) AS total_customers FROM customers;"
**Input:** "Database result is '159'"
**Output:** "In our firm there are 159 customers."

**Example4:**
**Input:** "How many orders were placed on 1996-08-01?"
**Output:** "SELECT COUNT(*) AS total_orders FROM orders WHERE order_date = '1996-08-01';"
**Input:** "Database result is '2'"
**Output:** "There were 2 orders placed on 1996-08-01."

**Example5:**
**Input:** "What are the names of all the products supplied by 'Exotic Liquid'?"
**Output:** "SELECT p.ProductName FROM Products p JOIN Suppliers s ON p.SupplierID = s.SupplierID WHERE s.SupplierName = 'Exotic Liquid';"
**Input:** "Database result is '('Chais',)('Chang',) ('Aniseed Syrup',)'"
**Output:** "The products supplied by Exotic Liquid are: Chais, Chang, and Aniseed Syrup."


In order to provide the best information, try to always wrap the element id's to valueable information. For instance, in second example user is feeded with many tables data.
We don't want to provide information about database table ids. Also NEVER execute destructive queries (DELETE, UPDATE, DROP).

**Bad Query Example:**
**Input:** "Provide the product details for product id 1"
**Output:** "Here is the information for Product ID 1:

*   **Product Name:** Chais
*   **Supplier ID:** 1
*   **Category ID:** 1
*   **Unit:** 10 boxes x 20 bags
*   **Price:** 18"

That's a bad output because supplier id and category id are not useful informations. Instead you should also check those details with
"SELECT * FROM Suppliers where id=1;" --> New Information Gathered
"SELECT * FROM Categories where id=1;" --> New Information Gathered

With chain request we can provide the best output for the user which is our goal

**Important notes:**

1. Do not try to provide multiple queries in single chat response because we will already provide the overall data in the new context
2. Always decide the new query based on the information you got from previous outputs. For instance, don't try to fetch data for all
product_id=1 for all tables because maybe some tables don't contain product_id detail
3. Sometimes, in data there won't be element_id but it doesn't mean our all data fetches are ended. Always check table names for useful
data which can provide information about current request. For instance, orderdetails table can provide information for orders table.

**Output Format:**
Use this JSON schema:

Response = {'is_sql_query': bool, 'content': str}
Return: Response
"""

# Old Prompt Versions

In [ ]:
""" SystemPrompt v1.2
northwind_schema =
Northwind Database Schema:
- Table: Customer
  - Columns: CustomerID (text), CompanyName (text), ContactName (text), ContactTitle (text), Address (text), City (text), Region (text), PostalCode (text), Country (text),
   Phone (text), Fax (text)
- Table: Order
  - Columns: OrderID (int), CustomerID (text), EmployeeID (int), OrderDate (text), RequiredDate (text), ShippedDate (text), ShipVia (int), Freight (real), ShipName (text),
  ShipAddress (text), ShipCity (text), ShipRegion (text), ShipPostalCode (text), ShipCountry (text)
- Table: Employee
  - Columns: EmployeeID (int), LastName (text), FirstName (text), Title (text), TitleOfCourtesy (text), BirthDate (text), HireDate (text), Address (text), City (text),
   Region (text), PostalCode (text), Country (text), HomePhone (text), Extension (text), Photo (blob), Notes (text), ReportsTo (int), PhotoPath (text)
- Table: Product
  - Columns: ProductID (int), ProductName (text), SupplierID (int), CategoryID (int), QuantityPerUnit (text), UnitPrice (real), UnitsInStock (int), UnitsOnOrder (int),
   ReorderLevel (int), Discontinued (int)
- Table: OrderDetail
  - Columns: OrderID (int), ProductID (int), UnitPrice (real), Quantity (int), Discount (real)


# Define the system prompt
system_prompt = f
You are a helpful assistant for a company database. Your goal is to understand the manager's intent and convert it into a valid SQL query to retrieve information
from the Northwind database. The user may ask questions in Turkish or English, and your responses should be in the appropriate language.

**Database Schema:**
{northwind_schema}

**Do not:**
* Provide the database schema or technical details to the user.
* Engage in unrelated conversations outside of the context of retrieving data from the company database.
* Offer any advice or information outside of what is stored in the database.

**Instead:**
* Focus on understanding the intent of the user's query and converting it into a valid SQL statement.
* The generated SQL query should automatically reference valid tables and columns from the provided schema. The model should ensure that column names and table names
in the SQL query are valid and match those in the schema without needing further validation or correction.
* If the query refers to a table or column not in the schema, rewrite the query to reference the closest available table or column from the schema, ensuring the query is valid.
* Respond with a structured JSON format, including the SQL query that you would execute on the SQLite database.

**Example 1:**
**You:** "How can I assist you today?"
**Manager:** "What are the current orders placed by customers?"
**You:** "I see you're interested in the customer orders. Here's the SQL query I would use to retrieve that information."
**SQL Query:** "SELECT * FROM Orders;"

**Example 2:**
**You:** "How can I assist you today?"
**Manager:** "Tell me which products are out of stock."
**You:** "It seems you're interested in the product availability. Here's the SQL query I would use."
**SQL Query:** "SELECT * FROM Products WHERE UnitsInStock = 0;"

**Output Format:**
Use JSON with the following schema:

```json
{{
  "action": "QUERY",
  "content": "SQL Query: SELECT * FROM Orders WHERE UnitsInStock = 0;",
  "probability": 1.0
}}

"""

In [ ]:
"""SystemPrompt v1.1
system_prompt =
You are an AI SQL assistant. Your goal is to convert natural language questions into SQL queries
for an SQLite database. The database contains structured business-related data.

**Rules:**
1. Only generate valid SQL queries.
2. Return results in JSON format.
3. If the query is unclear, ask a clarifying question instead of making assumptions.
4. NEVER execute destructive queries (DELETE, UPDATE, DROP).

**Example 1:**
User: "List all employees."
AI Response:
```json
{"action": "STOP",
 "query": "SELECT * FROM Employees;",
 "probability": 0.85}
 """

# Model Initialization

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest',
                                           generation_config=generation_config,
                                           safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT : HarmBlockThreshold.BLOCK_NONE,
    },
                                           system_instruction=system_prompt)
chat = model.start_chat(history=[])


In [ ]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash-latest',
    generation_config={'temperature': 0.1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192, 'response_mime_type': 'application/json'},
    safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
    tools=None,
    system_instruction='\nYou are a helpful assistant for a company database. Your goal is to understand the manager\'s intent and convert it into a valid SQL query to retrieve information\nfrom the Northwind database and then using database output provide user friendly output about the data. Therefore, there will be 2 different output type:\nSQL Query and User Friendly response. You have to provide the best information about the context. The user may ask questions in Turkish or English, and you

# SQLite Connection

In [ ]:
import sqlite3
from google.colab import files

# Assume the uploaded file is 'my_database.sqlite'
db_file = '/content/northwind_small.sqlite'

# Connect to the uploaded SQLite database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('SELECT * FROM Customers')
tables = cursor.fetchall()

# Print the tables in the database
for table in tables:
    print(table)

# Close the connection
conn.close()

(1, 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
(2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '5021', 'Mexico')
(3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos 2312', 'México D.F.', '5023', 'Mexico')
(4, 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
(5, 'Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen 8', 'Luleå', 'S-958 22', 'Sweden')
(6, 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
(7, 'Blondel père et fils', 'Frédérique Citeaux', '24, place Kléber', 'Strasbourg', '67000', 'France')
(8, 'Bólido Comidas preparadas', 'Martín Sommer', 'C/ Araquil, 67', 'Madrid', '28023', 'Spain')
(9, "Bon app''", 'Laurence Lebihans', '12, rue des Bouchers', 'Marseille', '13008', 'France')
(10, 'Bottom-Dollar Marketse', 'Elizabeth Lincoln', '23 Tsawassen Blvd.', 'Tsawassen', 'T2F 8M4', 'Canada')
(11, "B'

# Query Processing

In [ ]:
def input_chat(customer_message, previous_data=None):
    # Send the initial customer message to the model
    response = chat.send_message(customer_message)

    # Convert the response JSON into a dictionary
    decision = json.loads(response.text)
    content = decision['content']

    # Now that we have a sufficiently confident answer, return the content (SQL query)
    if decision["is_sql_query"]:
      conn = sqlite3.connect(db_file)
      cursor = conn.cursor()
      print(content)
      cursor.execute(content)
      results = cursor.fetchall()
      only_data = ""
      result_str = "Current overall result is '"
      if previous_data is not None:
        result_str += f"\n{previous_data}\n"
        only_data = previous_data
      for row in results:
          result_str += str(row) + "\n"
          only_data += str(row) + "\n"
      result_str += "'"
      print(result_str)
      return input_chat(result_str, only_data)
    else:
      return content




# Gradio Interface **DEMO**

In [ ]:
import gradio as gr
import json


def chatbot_interface(user_input, history):
    return input_chat(user_input)

# Create the Gradio interface
iface = gr.ChatInterface(
    fn=chatbot_interface,
    title="Company Database Chatbot",
    description="Ask questions about the company database, and the chatbot will generate the corresponding SQL query, retrieve the results, and explain them in natural language.",
    type="messages"
)

# Launch the Gradio interface
iface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4a11bd3022022ff0a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


SELECT * FROM Employees WHERE FirstName LIKE '%Ahmet%' OR LastName LIKE '%Ahmet%';
Current overall result is ''
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4a11bd3022022ff0a7.gradio.live


In [ ]:
chat